In [1]:
#installing tensorflow

#bellow command were executed on wsl terminal

#conda install -c conda-forge cudatoolkit=11.8.0
#python3 -m pip install nvidia-cudnn-cu11==8.6.0.163
#pip install --upgrade pip
#python3 -m pip install tensorflow==2.12.*

In [2]:
#installing packages
!python3 -m pip install -q -U watermark
!python3 -m pip install -U pip
!python3 -m pip install -U matplotlib
!python3 -m pip install scikit-learn
!python3 -m pip install pandas


/bin/bash: /home/bruno/miniconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/bruno/miniconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
/bin/bash: /home/bruno/miniconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.5/159.5 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 40.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.0/305.0 kB 75.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 46.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 42.9 MB/s eta 0:00:00a 0:00:01


In [3]:
#Imports

#Data visualization and manipulation
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

#Image handling
import os
import sklearn
from PIL import Image
from sklearn.model_selection import train_test_split

#Utility and calculation
import itertools
from sklearn.utils.multiclass import unique_labels
from sklearn.utils import shuffle

#Deep learning
import tensorflow
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, MaxPool2D
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.layers import Dropout, Activation
from tensorflow.keras.layers import BatchNormalization, GlobalAveragePooling2D, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
from keras.utils import to_categorical
from keras.applications import InceptionV3
from keras.applications.inception_v3 import preprocess_input

#Disabling Pandas and Matplotslib warnings
import sys
import warnings
import matplotlib.cbook
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
#warnings.filterwarnings("ignore", category=matplotlib.cbook.MatplotlibDeprecationWarning)

%matplotlib inline


2024-05-13 19:21:58.368763: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-13 19:21:59.347027: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:

#Package versions in this Jupyter notebook
%reload_ext watermark
%watermark -a "Bruno Castro - MBA USP ESALQ" --iversions

Author: Bruno Castro - MBA USP ESALQ

pandas    : 2.2.2
keras     : 2.12.0
sys       : 3.9.19 (main, May  6 2024, 19:43:03) 
[GCC 11.2.0]
numpy     : 1.24.3
matplotlib: 3.8.4
tensorflow: 2.12.1
sklearn   : 1.4.2
PIL       : 10.3.0



In [5]:
#List hardware available
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17278024958513796389
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5863636992
locality {
  bus_id: 1
  links {
  }
}
incarnation: 6896084870553769036
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3060 Ti, pci bus id: 0000:06:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


2024-05-13 19:22:18.015716: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-13 19:22:18.158717: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-13 19:22:18.158802: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-13 19:22:18.328714: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-13 19:22:18.328837: I tensorflow/compile

In [6]:
import tensorflow as tf
print("Number of available GPUs: ", len(tf.config.experimental.list_physical_devices('GPU')))

Number of available GPUs:  1


2024-05-13 19:22:53.128636: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-13 19:22:53.128875: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-13 19:22:53.128952: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.


In [7]:
#List code of each GPU
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]